# Amazon SageMaker JumpStart で学習したモデルを使ってローカルで推論する

Amazon SageMaker JumpStart で Tensorflow Hub や PyTorch Hub のモデルを転移学習させたあと、Deploy ボタンをクリックすることで学習みモデルがデプロイされた推論エンドポイントを立ち上げることができます。しかし、ユースケースによっては推論エンドポイントではなくローカルPCやその他の環境で学習済みモデルを使って推論を実行したい場合もあります。

このノートブックでは、Amazon SageMaker JumpStart で学習したモデルを使って推論を実行します。PyTorch の VGG モデルを使う場合の方法を説明していますが、他のモデルや Tensorflow のモデルも同様の方法で推論が可能です（推論を実行するコードはフレームワークごとの書き方に変える必要があります）。

## 学習済みモデルのダウンロードと解凍

Amazon SageMaker の学習ジョブの機能を使ってモデルを学習すると、学習済みモデルは tar.gz で圧縮されて Amazon S3 に保存されます。まずは、S3 に保存された学習済みモデルをノートブックインスタンスにダウンロードします。今回は AWS CLI を使いますが、boto3 などを使っても OK です。

**以下のコマンドの model.tar.gz のパスを、所望の学習済みモデルが保存された S3 パスに書き換えてから実行してください。**

In [ ]:
!aws s3 cp s3://bucket/job-name/output/model.tar.gz ./
!tar zvxf model.tar.gz

## モデルのロード

model.tar.gz を解凍すると model.pt が作成されるので、それをロードします。

In [ ]:
import torch

model = torch.load('model.pt')
model.eval()

## 分類クラス情報の取得

model.tar.gz を解凍すると、分類クラスが記載された class_label_to_prediction_index.json が作成されるので、これを読み込んで分類クラスの情報を取得します。

In [ ]:
import json

json_open = open('class_label_to_prediction_index.json', 'r')
json_load = json.load(json_open)
class_names = []
for v in json_load.keys():
    class_names.append(v)
class_names

## 前処理パラメタの取得

機械学習モデルに入力する画像には前処理によって正規化されています。推論を実行する際にも学習時と同じ正規化のパラメタを使用する必要があります。そのため、学習の際に使用したソースコードを取得し、前処理に関するパラメタを確認します。

モデルの学習に使用されたソースコードは、学習ジョブの詳細画面の「ハイパーパラメータ」の項目に、`sagemaker_submit_directory` として記録されたパスに保存されています。たとえば、`s3://bucket/job-name/source/sourcedir.tar.gz` と書かれています。この tar.gz ファイルをダウンロードして解凍します。

**以下のコマンドの sourcedir.tar.gz のパスを、所望のソースコードが保存された S3 パスに書き換えてから実行してください。**

In [ ]:
!aws s3 cp s3://bucket/job-name/source/sourcedir.tar.gz ./
!tar zvxf sourcedir.tar.gz

tar.gz ファイルを解凍すると、今回使用した VGG モデルの場合、`constants/constants.py` というファイルが作成されます。このファイルに、画像の前処理で使用するパラメタ `RANDOM_RESIZED_CROP`, `NORMALIZE_MEAN`, `NORMALIZE_STD` が記載されているので、以下のパラメタをコピーします。

In [ ]:
RANDOM_RESIZED_CROP = 224
NORMALIZE_MEAN = [0.485, 0.456, 0.406]
NORMALIZE_STD = [0.229, 0.224, 0.225]

コピーしたパラメタを使って、画像を変換するための transforms.Compose を作成します。

In [ ]:
from torchvision import transforms

transform = transforms.Compose(
    [
    transforms.Resize(RANDOM_RESIZED_CROP),
    transforms.CenterCrop(RANDOM_RESIZED_CROP),
    transforms.ToTensor(),
    transforms.Normalize(NORMALIZE_MEAN, NORMALIZE_STD),
    ]
)

## 推論の実行

**下準備として、推論で使用する画像を images フォルダに保存してから、以降のセルを実行してください。**

images フォルダの中のファイルを DataLoader で読み込みます。

In [ ]:
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import numpy as np
import glob

device = 'cpu'

class ImageFolder(Dataset):
    def __init__(self, img_dir, transform):
        # 画像ファイルのパス一覧を取得する。
        self.img_paths = glob.glob(img_dir + '/**')
        self.transform = transform

    def __getitem__(self, index):
        path = self.img_paths[index]
        img = Image.open(path)
        inputs = self.transform(img)

        return {"image": inputs, "path": path}

    def __len__(self):
        return len(self.img_paths)


# Dataset を作成する。
dataset = ImageFolder("images", transform)
# DataLoader を作成する。
dataloader = DataLoader(dataset, batch_size=10)

images フォルダ内の画像を使って推論を実行します。

In [ ]:
from IPython import display
from torch.nn import functional as F

for batch in dataloader:
    inputs = batch["image"].to(device)
    outputs = model(inputs)

    batch_probs = F.softmax(outputs, dim=1)

    batch_probs, batch_indices = batch_probs.sort(dim=1, descending=True)

    for probs, indices, path in zip(batch_probs, batch_indices, batch["path"]):
        display.display(display.Image(path, width=RANDOM_RESIZED_CROP))
        print(f"path: {path}")
        for k in range(min(len(class_names), 3)):
            print(f"Top-{k + 1} {probs[k]:.2%} {class_names[indices[k]]}")